In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import os
from sklearn.metrics import accuracy_score

# Define dataset class
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [6]:
import os
import shutil
from tqdm import tqdm

base_dir = "../Datasets/SIPaKMeD"
output_dir = "../Outputs/sipakmed_combine"
folders = ["im_Dyskeratotic", "im_Koilocytotic", "im_Metaplastic", "im_Parabasal", "im_Superficial-Intermediate"]

os.makedirs(output_dir, exist_ok=True)

label_mapping = {folder: i for i, folder in enumerate(folders)}

for folder in folders:
    class_dir = os.path.join(base_dir, folder, "CROPPED")
    
    if os.path.exists(class_dir):
        for file in tqdm(os.listdir(class_dir), desc=f"Processing {folder}"):
            if file.endswith(".bmp"): 
                old_path = os.path.join(class_dir, file)
                new_filename = f"{label_mapping[folder]}__{file.split('.')[0]}.bmp"
                new_path = os.path.join(output_dir, new_filename)
                shutil.copy(old_path, new_path)

print("Organisation et renommage des fichiers terminés ! ✅")


Processing im_Superficial-Intermediate: 100%|██████████| 2493/2493 [00:00<00:00, 5162.88it/s]

Organisation et renommage des fichiers terminés ! ✅


In [7]:
import os
from sklearn.model_selection import train_test_split

image_folder = "../Outputs/sipakmed_combine"

image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.bmp')]
labels = [int(img.split('__')[0]) for img in os.listdir(image_folder) if img.endswith('.bmp')]  # Extraction du label

print(f"Total images: {len(image_paths)}")
print(f"Total labels: {len(labels)}")

Total images: 4049
Total labels: 4049


In [9]:
split_ratios = {"train": 0.75, "cal": 0.1125, "val": 0.0375, "test": 0.10}

train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=1 - split_ratios["train"], stratify=labels, random_state=42
)

remaining_ratio = 1 - split_ratios["train"]
cal_ratio = split_ratios["cal"] / remaining_ratio
val_ratio = split_ratios["val"] / remaining_ratio
test_ratio = split_ratios["test"] / remaining_ratio

cal_paths, temp_paths, cal_labels, temp_labels = train_test_split(
    temp_paths, temp_labels, test_size=(val_ratio + test_ratio), stratify=temp_labels, random_state=42
)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=(test_ratio / (val_ratio + test_ratio)), stratify=temp_labels, random_state=42
)

print(f"Train: {len(train_paths)}, Cal: {len(cal_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}")

Train: 3036, Cal: 455, Val: 152, Test: 406


In [10]:

dataset_train = ImageDataset(train_paths, train_labels, transform=transform)
dataset_val = ImageDataset(val_paths, val_labels, transform=transform)
dataset_cal = ImageDataset(cal_paths, cal_labels, transform=transform)
dataset_test = ImageDataset(test_paths, test_labels, transform=transform)

dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=32, shuffle=False)
dataloader_cal = DataLoader(dataset_cal, batch_size=32, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)


In [ ]:

# Load ResNet50 model
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Identity() 
resnet.eval()
resnet.to('cuda' if torch.cuda.is_available() else 'cpu')

# Feature extraction function
def extract_features(dataloader):
    features, labels_list = [], []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Extracting features"):
            images = images.to(device)
            feat = resnet(images)
            features.append(feat.cpu().numpy())
            labels_list.extend(labels.numpy())
    return np.vstack(features), np.array(labels_list)

# Extract features for train, val, and test
features_train, labels_train = extract_features(dataloader_train)
features_val, labels_val = extract_features(dataloader_val)
features_cal, labels_cal = extract_features(dataloader_cal)
features_test, labels_test = extract_features(dataloader_test)


Extracting features: 100%|██████████| 13/13 [00:04<00:00,  2.94it/s]


In [28]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
set_seed()

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Define MLP classifier
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Training the MLP
input_dim = features_train.shape[1]
num_classes = len(set(labels_train))
mlp = MLPClassifier(input_dim, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001, weight_decay=1e-4)

def get_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield torch.tensor(X[i:i+batch_size], dtype=torch.float32), torch.tensor(y[i:i+batch_size], dtype=torch.long)

# Training loop
num_epochs = 25
best_val_loss = float('inf')

for epoch in range(num_epochs):
    mlp.train()
    running_loss = 0.0
    for X_batch, y_batch in get_batches(features_train, labels_train, batch_size=32):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = mlp(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation phase
    mlp.eval()
    val_loss = 0.0
    val_preds, val_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in get_batches(features_val, labels_val, batch_size=32):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = mlp(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            val_targets.extend(y_batch.cpu().numpy())
    
    val_accuracy = accuracy_score(val_targets, val_preds)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(mlp.state_dict(), "best_mlp_classifier.pth")

# Testing phase
mlp.load_state_dict(torch.load("best_mlp_classifier.pth"))
mlp.eval()
test_preds, test_targets = [], []

with torch.no_grad():
    for X_batch, y_batch in get_batches(features_test, labels_test, batch_size=32):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = mlp(X_batch)
        test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        test_targets.extend(y_batch.cpu().numpy())




Epoch 1/25, Loss: 46.8372, Val Loss: 1.3598, Val Acc: 0.8816
Epoch 2/25, Loss: 21.4198, Val Loss: 1.2374, Val Acc: 0.9145
Epoch 3/25, Loss: 17.9827, Val Loss: 1.0701, Val Acc: 0.9408
Epoch 4/25, Loss: 17.7786, Val Loss: 0.9791, Val Acc: 0.9342
Epoch 5/25, Loss: 14.4647, Val Loss: 1.2757, Val Acc: 0.9013
Epoch 6/25, Loss: 11.7573, Val Loss: 1.3027, Val Acc: 0.8882
Epoch 7/25, Loss: 10.1886, Val Loss: 1.0554, Val Acc: 0.9342
Epoch 8/25, Loss: 9.1647, Val Loss: 0.9319, Val Acc: 0.9408
Epoch 9/25, Loss: 7.8848, Val Loss: 1.5678, Val Acc: 0.9276
Epoch 10/25, Loss: 8.9252, Val Loss: 1.3154, Val Acc: 0.9342
Epoch 11/25, Loss: 8.4252, Val Loss: 1.1965, Val Acc: 0.9539
Epoch 12/25, Loss: 8.7049, Val Loss: 0.9013, Val Acc: 0.9539
Epoch 13/25, Loss: 8.4871, Val Loss: 1.0519, Val Acc: 0.9408
Epoch 14/25, Loss: 7.7927, Val Loss: 1.0113, Val Acc: 0.9408
Epoch 15/25, Loss: 7.0666, Val Loss: 0.9268, Val Acc: 0.9408
Epoch 16/25, Loss: 6.5883, Val Loss: 1.2564, Val Acc: 0.9276
Epoch 17/25, Loss: 6.0760,

/tmp/ipykernel_483584/1456636576.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mlp.load_state_dict(torch.load("best_mlp_classifier.pth"))


In [30]:
test_accuracy = accuracy_score(test_targets, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9236
